Os dados são totalmente fictícios. Eu usei o gerador de dados do Python (biblioteca Faker) para criar informações como nomes, e-mails, números de telefone e endereços de forma aleatória, sem qualquer ligação com dados reais.

Portanto,pode ser ultilizado livremente para fins de estudo e prática sem preocupação com privacidade ou dados reais. 


Documentação:<br>
https://learn.microsoft.com/pt-br/azure/databricks/ingestion/cloud-object-storage/auto-loader/

Script de Exemplo <br>
https://learn.microsoft.com/pt-br/azure/databricks/ingestion/cloud-object-storage/auto-loader/unity-catalog

In [0]:
%python
# Definir o caminho do arquivo e o caminho do checkpoint
path = 'dbfs:/FileStore/Clientes Auto Loader' # arquivos parquet de origem 
checkpoint_path = 'dbfs:/FileStore/Clientes Auto Loader checkpoint/checkpoint'
# Ler o arquivo Parquet usando Auto Loader

df = (spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "parquet")
  .option("cloudFiles.schemaLocation", checkpoint_path)
  .load(path)
)
  

# Escrever o DataFrame em uma tabela Delta
(df.writeStream
  .option("checkpointLocation", checkpoint_path)
  .table("clientes_auto_loader")
)


Exemplo de arquivo CSV e json - Simulado , deve adptar para o seu recurso  e muitas vezes deve inferir o schema manual
 em arquivos problematicos

In [0]:
%sql
select * from clientes_auto_loader limit 10

In [0]:
%sql 
describe detail clientes_auto_loader

In [0]:
%sql
describe extended clientes_auto_loader

In [0]:
%sql
describe history clientes_auto_loader

In [0]:
%sql
select count(*) from clientes_auto_loader

In [0]:
%python
#Exemplo CSV

# Definir o caminho do arquivo e o caminho do checkpoint
file_path = "dbfs:/FileStore/Clientes Auto Loader"
checkpoint_path = "dbfs:/FileStore/Clientes Auto Loader/checkpoint"

# Ler o arquivo CSV usando Auto Loader
df = (spark.readStream
      .format("cloudFiles")
      .option("cloudFiles.format", "csv")
      .option("header", "true")
      .option("delimiter", ",")
      .option("cloudFiles.schemaLocation", checkpoint_path)
      .load(file_path))

# Escrever o DataFrame em uma tabela Delta
(df.writeStream
    .option("checkpointLocation", checkpoint_path)
    .outputMode("append")
    .option("mergeSchema", "true")
    .table("clientes_auto_loader"))

In [0]:
%python
#Exemplo Json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Define the schema for the JSON files
schema = StructType([
    StructField("field1", StringType(), True),
    StructField("field2", IntegerType(), True),
    # Add other fields as necessary
])

# Define the file path and checkpoint path
file_path = "dbfs:/FileStore/Clientes Auto Loader"
checkpoint_path = "dbfs:/FileStore/Clientes Auto Loader checkpoint/checkpoint"

# Read the JSON files using Auto Loader with the specified schema
df = (spark.readStream
      .format("cloudFiles")
      .option("cloudFiles.format", "json")
      .option("cloudFiles.schemaLocation", checkpoint_path)
      .schema(schema)
      .load(file_path))

# Write the DataFrame to a Delta table
(df.writeStream
    .option("checkpointLocation", checkpoint_path)
    .outputMode("append")
    .table("clientes_auto_loader"))

Deletar pastas e tabelas


In [0]:
%sql 
Drop table clientes_auto_loader

In [0]:
%python
dbutils.fs.rm('dbfs:/FileStore/Clientes Auto Loader checkpoint', recurse=True)
dbutils.fs.rm('dbfs:/FileStore/Clientes Auto Loader', recurse=True)
